# Configuration

NOTES: The warnings after the import are referred to the fact that Tensorflow 2.x versions are built to directly look for a GPU in the system. The warning can be forgot if you are not going to use the GPU. 

In [1]:
!source myenv/bin/activate

In [2]:
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set_style('whitegrid')
import IPython.display as ipd
import librosa.display
import numpy as np
import pickle
import scipy
import ipywidgets
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold


from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# from livelossplot import PlotLossesKeras
tf.config.list_physical_devices('GPU')

2021-09-10 11:11:04.093670: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-10 11:11:04.093691: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-10 11:11:04.910051: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-10 11:11:04.910609: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-10 11:11:04.983024: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-10 11:11:04.983049: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /pr

[]

# Utils

# Compute dataframes for datasets and split in Train, Val, Test 

In [3]:
main_path = '/media/helemanc/OS/Users/i2CAT/Desktop/Datasets SER/'
TESS = os.path.join(main_path, "tess/TESS Toronto emotional speech set data/") 
RAV = os.path.join(main_path, "ravdess-emotional-speech-audio/audio_speech_actors_01-24")
SAVEE = os.path.join(main_path, "savee/ALL/")
CREMA = os.path.join(main_path, "creamd/AudioWAV/")

In [4]:
males = [1,
5,
11,
14,
15,
16,
17,
19,
22,
23,
26,
27,
31,
32,
33,
34,
35,
36,
38,
39,
41,
42,
44,
45,
48,
50,
51,
57,
59, 
62, 
64,
65, 
66,
67,
68,
69,
70,
71,
77, 
80, 
81, 
83, 
85, 
86, 
87,
88, 
90]

In [5]:
females = [ 2,
3,
4,
6,
7,
8,
9,
10,
12,
13,
18,
20,
21,
24,
25,
28,
29,
30,
37,
40,
43,
46,
47,
49,
52,
53,
54,
55,
56, 
58, 
60,
61,
63,
72, 
73, 
74, 
75, 
76, 
78, 
79, 
82, 
84, 
89, 
91]

In [6]:
crema_directory_list = os.listdir(CREMA)

file_emotion = []
file_path = []
actors = []
gender = []




for file in crema_directory_list:

    # storing file emotions
    part=file.split('_')
    
    # use only high intensity files
    if "HI" in part[3] :
        actor = part[0][2:]
        actors.append(actor)
        if int(actor) in males:
            gender.append('male')
        else: 
            gender.append('female')
    
        # storing file paths
        file_path.append(CREMA + file)
        if part[2] == 'SAD':
            file_emotion.append('sadness')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['emotion_label'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['path'])
actors_df = pd.DataFrame(actors, columns=['actors'])
gender_df = pd.DataFrame(gender, columns=['gender'])                      
Crema_df = pd.concat([emotion_df, actors_df, gender_df, path_df], axis=1)
Crema_df.head()

,emotion_label,actors,gender,path
0,happy,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
1,sadness,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,angry,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
3,disgust,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,fear,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [7]:
Crema_df.shape

(455, 4)

In [8]:
actor_files = {}

for index, row in Crema_df.iterrows():
    actor = row['actors']
    if actor not in actor_files.keys(): 
        actor_files[actor] = 1
    else: 
        actor_files[actor]+=1

In [9]:
actor_files

{'91': 5,
 '90': 5,
 '89': 5,
 '88': 5,
 '87': 5,
 '86': 5,
 '85': 5,
 '84': 5,
 '83': 5,
 '82': 5,
 '81': 5,
 '80': 5,
 '79': 5,
 '78': 5,
 '77': 5,
 '76': 5,
 '75': 5,
 '74': 5,
 '73': 5,
 '72': 5,
 '71': 5,
 '70': 5,
 '69': 5,
 '68': 5,
 '67': 5,
 '66': 5,
 '65': 5,
 '64': 5,
 '63': 5,
 '62': 5,
 '61': 5,
 '60': 5,
 '59': 5,
 '58': 5,
 '57': 5,
 '56': 5,
 '55': 5,
 '54': 5,
 '53': 5,
 '52': 5,
 '51': 5,
 '50': 5,
 '49': 5,
 '48': 5,
 '47': 5,
 '46': 5,
 '45': 5,
 '44': 5,
 '43': 5,
 '42': 5,
 '41': 5,
 '40': 5,
 '39': 5,
 '38': 5,
 '37': 5,
 '36': 5,
 '35': 5,
 '34': 5,
 '33': 5,
 '32': 5,
 '31': 5,
 '30': 5,
 '29': 5,
 '28': 5,
 '27': 5,
 '26': 5,
 '25': 5,
 '24': 5,
 '23': 5,
 '22': 5,
 '21': 5,
 '20': 5,
 '19': 5,
 '18': 5,
 '17': 5,
 '16': 5,
 '15': 5,
 '14': 5,
 '13': 5,
 '12': 5,
 '11': 5,
 '10': 5,
 '09': 5,
 '08': 5,
 '07': 5,
 '06': 5,
 '05': 5,
 '04': 5,
 '03': 5,
 '02': 5,
 '01': 5}

In [10]:
count_males = 0 
count_females = 0 
male_list = []
for index, row in Crema_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1

In [11]:
count_males, count_females

(235, 220)

Since there are more males than females we will remove randomly 3 male actors (since there are exactly 5 audio files per actor)

In [12]:
import random 
random.seed(42)
males_to_remove = random.sample(male_list, 3)
males_to_remove

['17', '80', '88']

In [13]:
new_df = []
for index, row in Crema_df.iterrows(): 
    if row['actors'] not in males_to_remove: 
        new_df.append(row)

In [14]:
CREMA_df = pd.DataFrame(new_df)

In [15]:
for index, row in CREMA_df.iterrows(): 
    if row['actors'] == '17': 
        print("Elements not removed")

In [16]:
count_males = 0 
count_females = 0 
male_list = []
female_list = []
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1
        if actor not in female_list: 
            female_list.append(actor)

In [17]:
count_males, count_females

(220, 220)

In [18]:
len(female_list)

44

In [19]:
len(male_list)

44

In [20]:
CREMA_train = []
CREMA_val = []
CREMA_test = []

In [21]:
females_train = random.sample(female_list, 32)
males_train = random.sample(male_list, 32)

# remove the elements assigned to train 
for element in females_train:
    if element in female_list:
        female_list.remove(element)
        
for element in males_train:
    if element in male_list:
        male_list.remove(element)

         
females_val = random.sample(female_list, 6) 
males_val = random.sample(male_list, 6) 

# remove the elements assigned to val
for element in females_val:
    if element in female_list:
        female_list.remove(element)
        
for element in males_val:
    if element in male_list:
        male_list.remove(element)
        
females_test = random.sample(female_list, 6) 
males_test = random.sample(male_list, 6)        

In [22]:
females_train, males_train, females_val, males_val, females_test, males_test

(['54',
  '56',
  '58',
  '74',
  '76',
  '13',
  '78',
  '29',
  '84',
  '89',
  '09',
  '60',
  '04',
  '55',
  '52',
  '91',
  '02',
  '07',
  '46',
  '49',
  '37',
  '10',
  '20',
  '75',
  '21',
  '53',
  '06',
  '28',
  '18',
  '63',
  '30',
  '03'],
 ['57',
  '69',
  '65',
  '45',
  '77',
  '81',
  '41',
  '15',
  '44',
  '23',
  '59',
  '86',
  '34',
  '01',
  '85',
  '66',
  '31',
  '33',
  '05',
  '48',
  '50',
  '67',
  '51',
  '22',
  '36',
  '87',
  '71',
  '39',
  '42',
  '11',
  '32',
  '14'],
 ['43', '61', '40', '47', '73', '24'],
 ['62', '68', '64', '83', '70', '26'],
 ['08', '79', '12', '25', '72', '82'],
 ['16', '19', '38', '35', '27', '90'])

In [23]:
train = females_train + males_train 
val = females_val + males_val 
test = females_test + males_test

In [24]:
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if actor in train: 
        CREMA_train.append(row)
    elif actor in val: 
        CREMA_val.append(row)
    else:
        CREMA_test.append(row)

In [25]:
CREMA_train = pd.DataFrame(CREMA_train) 
CREMA_val = pd.DataFrame(CREMA_val) 
CREMA_test = pd.DataFrame(CREMA_test)

In [65]:
CREMA_train.shape, CREMA_val.shape, CREMA_test.shape

((320, 4), (60, 4), (60, 4))

In [26]:
CREMA_train = CREMA_train.reset_index(drop=True) 
CREMA_val = CREMA_val.reset_index(drop = True) 

In [27]:
df_train = CREMA_train 
df_val = CREMA_val 
df_test = CREMA_test

# Create Noise Files

In [28]:
from pydub import AudioSegment
import random 
from pydub.utils import make_chunks


def create_noise_files(df_train, df_val, df_test): 
    
    '''
    Apply noise only on training files, so double the number of training files and keep 
    validation and test the same
    '''
    path_noise_sound_1 = '/home/helemanc/Desktop/Binary_Model/noise_sounds/freight_train.wav'
    path_noise_sound_2 = '/home/helemanc/Desktop/Binary_Model/noise_sounds/inside_train.wav'
    path_noise_sound_3 = '/home/helemanc/Desktop/Binary_Model/noise_sounds/small_crowd.wav'
    
    path_noise_dataset_train = '/home/helemanc/Desktop/Binary_Model/noise_datasets/crema/train'
    #path_noise_dataset_val = '/home/helemanc/Desktop/Binary_Model/noise_datasets/ravdess/val'
    #path_noise_dataset_test = '/home/helemanc/Desktop/Binary_Model/noise_datasets/ravdess/test'
    

    #df_list = [df_train, df_val, df_test]
    #count_df = 0 
    
    train_emotions = []
    train_genders = []
    train_paths = []
    
    #val_emotions = []
    #val_genders = []
    #val_paths = []
    
    #test_emotions = []
    #test_genders = []
    #test_paths = []
    
    #for df in df_list: 
        
    for index, row in tqdm(df_train.iterrows()): 
        path = row['path']
        sound1 = AudioSegment.from_file(path)
        samples, sr = librosa.load(path, res_type='kaiser_fast', sr=16000)
        duration = librosa.get_duration(y = samples, sr = sr)

        # pick a noise sound file randomly 
        noise_list = [path_noise_sound_1, path_noise_sound_2, path_noise_sound_3]
        random_noise = random.choice(noise_list) 

        lower_volume = 0 

        # adjust volume to not cover the voice of the audio file 
        # warning: different levels of dB need to be calibrate for each dataset 
        '''
        if random_noise == path_noise_sound_1: 
            lower_volume = 40
        elif random_noise == path_noise_sound_2: 
            lower_volume = 25 
        else: 
            lower_volume = 40
        '''

        # other strategy: 
        # compute db of both files, compute the difference, and lower the volume of the file to make it 
        # a bit lower than the original file -almost equal- 

        sound2 = AudioSegment.from_file(random_noise)

        # make chunks of duration equal to the audio file 
        chunk_length_ms = duration*1000 #ms
        chunks = make_chunks(sound2, chunk_length_ms) 

        # pick a random chunk 
        random_chunk = random.choice(chunks)
        difference = random_chunk.dBFS - sound1.dBFS

        abs_difference = abs(difference)

        lower = random_chunk - abs_difference - 2

        # lower the volume of the noise file to be overlayed with the voice_sound 
        #lower = random_chunk - lower_volume

        combined = sound1.overlay(lower)

        parts = path.split('/')
        fname = parts[-1]
        
        new_path = path_noise_dataset_train + '/' + fname 

        train_emotions.append(row['emotion_label'])
        train_genders.append(row['gender'])
        train_paths.append(new_path)

        '''
        if count_df == 0: 
            new_path = path_noise_dataset_train + '/' + fname 

            train_emotions.append(row['emotion_label'])
            train_genders.append(row['gender'])
            train_paths.append(new_path)

        elif count_df == 1: 
            new_path = path_noise_dataset_val + '/' + fname

            val_emotions.append(row['emotion_label'])
            val_genders.append(row['gender'])
            val_paths.append(new_path)

        elif count_df == 2:
            new_path = path_noise_dataset_test + '/' + fname          

            test_emotions.append(row['emotion_label'])
            test_genders.append(row['gender'])
            test_paths.append(new_path)
        '''
        combined.export(new_path, format= 'wav')

    #count_df +=1

    df_train_noise = pd.DataFrame([train_emotions, train_genders, train_paths]).T
    df_train_noise.columns = ['emotion_label', 'gender', 'path']
    
    #df_val_noise = pd.DataFrame([val_emotions, val_genders, val_paths]).T
    #df_val_noise.columns = ['emotion_label', 'gender', 'path']
    
    #df_test_noise = pd.DataFrame([test_emotions, test_genders, test_paths]).T
    #df_test_noise.columns = ['emotion_label', 'gender', 'path']

    df_train_combined = pd.concat([df_train, df_train_noise])
    df_train_combined.reset_index(drop=True, inplace=True)
    
    #df_val_combined = pd.concat([df_val, df_val_noise])
    #df_val_combined.reset_index(drop=True, inplace=True)
                                   
    #df_test_combined = pd.concat([df_test, df_test_noise])
    #df_test_combined.reset_index(drop=True, inplace=True)
    
    return df_train_combined, df_val, df_test
# have to save df 

In [29]:
new_df_train, new_df_val, new_df_test = create_noise_files(df_train, df_val, df_test)

320it [00:00, 351.24it/s]


In [30]:
new_df_train.shape, new_df_val.shape, new_df_test.shape

((640, 4), (60, 4), (60, 4))

## Save dataframes

In [31]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/crema"

In [32]:
new_df_train.to_csv(os.path.join(preprocess_path,"df_train.csv"), index=False)

In [33]:
new_df_val.to_csv(os.path.join(preprocess_path,"df_val.csv"), index=False)

In [34]:
new_df_test.to_csv(os.path.join(preprocess_path,"df_test.csv"), index=False)